In [ ]:
!pip3 install pyspark weaviate-client

In [ ]:
!pip show pyspark

In [ ]:
!sudo apt-get install scala

In [ ]:
!wget https://repo1.maven.org/maven2/io/weaviate/spark-connector_2.13/1.2.4/spark-connector_2.13-1.2.4.jar

In [31]:
!mv /content/spark-connector_2.13-1.2.4.jar /usr/local/lib/python3.10/dist-packages/pyspark/jars

In [19]:
from pyspark.sql import SparkSession
import os

spark = (
    SparkSession.builder.config(
        "spark.jars",
        "weaviate-spark-connector-assembly-v0.1.2.jar" #specify the spark connector JAR
    )
    .master("local[*]")
    .appName("weaviate")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

In [ ]:
!wget https://storage.googleapis.com/sphere-demo/sphere.100k.jsonl.tar.gz

In [8]:
!gunzip /content/sphere.100k.jsonl.tar.gz

In [9]:
!tar -xvf /content/sphere.100k.jsonl.tar

sphere.100k.jsonl


In [20]:
df = spark.read.load("sphere.100k.jsonl", format="json")

In [14]:
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(embedded_options=EmbeddedOptions())

embedded weaviate is already listing on port 6666


In [16]:
client.schema.delete_all()

client.schema.create_class(
    {
        "class": "Sphere",
        "properties": [
            {
                "name": "raw",
                "dataType": ["string"]
            },
            {
                "name": "sha",
                "dataType": ["string"]
            },
            {
                "name": "title",
                "dataType": ["string"]
            },
            {
                "name": "url",
                "dataType": ["string"]
            },
        ],
    }
)

Embedded weaviate wasn't listening on port 6666, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 16160


In [21]:
df.limit(1500).withColumnRenamed("id", "uuid").write.format("io.weaviate.spark.Weaviate") \
    .option("batchSize", 200) \
    .option("scheme", "http") \
    .option("host", "localhost:8080") \
    .option("id", "uuid") \
    .option("className", "Sphere") \
    .option("vector", "vector") \
    .mode("overwrite").save()

Py4JJavaError: ignored